In [1]:
import pandas as pd
import numpy as np
import pickle


## LSOA level

#### Census Data

In [2]:
census_all = pd.read_excel('data/lsoa/lsoa-data.xls',sheet='censusdata')

In [3]:
census = (census_all[['Codes','Population','Area (Hectares)','Median Price (£)','Sales','Employment Rate','Unemployment Rate','Mean Annual Household Income estimate (£)']]
          .rename(columns={'Codes':'lsoa_code','Area (Hectares)':'Area','Median Price (£)':'Median Price','Mean Annual Household Income estimate (£)':'Mean Household Income'}))

In [4]:
for col in census.columns[2:]:
    census[col] = census[col].astype('float')

In [5]:
# with open('pickle/census_data.pkl', 'wb') as picklefile:
#     pickle.dump(census,picklefile)

#### Index of Deprivation

In [8]:
imd2010 = (pd.read_excel('data/lsoa/indices-of-deprivation-2010.xls', sheet_name='IMD 2010')
           .rename(columns={'LSOA':'lsoa_code'})
           .drop(['Ward code','Ward Name','LA CODE','LA NAME','GOR CODE','GOR NAME'],axis=1))


In [9]:
for col in imd2010.columns[1:]:
    imd2010[col] = imd2010[col].astype('float')

In [10]:
# with open('pickle/imd_lsoa_2010.pkl', 'wb') as picklefile:
#     pickle.dump(imd2010,picklefile)

## Borough Level

In [12]:
imd2010 = (pd.read_excel('data/lsoa/indices-of-deprivation-2010.xls', sheet_name='IMD 2010')
           .rename(columns={'LSOA':'lsoa_code'}))

In [13]:
with open('pickle/census_data.pkl', 'rb') as picklefile:
    census = pickle.load(picklefile)

pop = census[['lsoa_code','Population']]

In [14]:
imd2010 = (imd2010
           .merge(pop, on='lsoa_code')
           .rename(columns={'LA NAME':'borough'})
           .drop(['Ward code', 'Ward Name', 'LA CODE', 'GOR CODE', 'GOR NAME','lsoa_code'],axis=1))

In [15]:
# Creating a weighted mean function to average LSOA data into borough level
wm = lambda x: np.average(x, weights=imd2010.loc[x.index, "Population"])



In [16]:
imd2010_borough = imd2010.groupby(['borough']).agg(wm)

In [17]:
# with open('pickle/imd_borough_2010.pkl', 'wb') as picklefile:
#     pickle.dump(imd2010_borough,picklefile)

In [18]:
census_borough = pd.read_excel('data/borough/london-borough-profiles.xlsx', sheet_name='Data')

In [20]:
# with open('pickle/census_borough.pkl', 'wb') as picklefile:
#     pickle.dump(imd2010_borough,picklefile)